# InvokeAI in Google Colab

## This version for 5.6.0+, This has had a major update since the PRE-5.5.0 version. Images or models will not migrate between the old and the new versions of this colab.

###Introduction

This is a tool to use Google colab to run the AI image generation tool: Invokeai (https://invoke-ai.github.io/InvokeAI/).
This automatically builds itself, It can connect to Google drive to save your images.
It also has the option of running from Google Drive, This takes about 2GB + Models of Google Drive space.

Make sure to enable GPU This should be on by default, but the setting can be found in the menu under: Edit > Notebook Settings > Hardware accelerator > GPU

To start, Click "Runtime" > "Run All". Alternaivly you can click the "play" button on each step below one after the other, No need to wait for the previous steps to finish as they will join a queue.

This uses the "Low VMRAM move" included with InvokeAI 5.5.0+

In [ ]:
#@title 1. Configuration { display-mode: "form" }
#@markdown #Instance Type
#@markdown **Google_Drive** = Stores Models, Images and the database in your Google drive, the space it takes up is hevily based on the type and number of models that you are using. <br>
#@markdown **Temporary** = Everything is stored in the runtime and is removed when the runtime ends or crashes, make sure to download your images! <br>
Type = "Google_Drive" #@param ['Google_Drive','Temporary'] {type:"string"}

#@markdown ---

#@markdown #Connection Type.
#@markdown **NGROK**: (Recomended) Highly stable but needs a little setting up, An NGROK token is required, sign up for free and get one here: https://dashboard.ngrok.com/get-started/your-authtoken Once you have the token, please put it in below.<br>
#@markdown **NGROK_APT**: An aternate version of NGROK that runs as a Linux service rather than a python service.<br>
#@markdown **Localtunnel**: Stable once connected, but sometimes has issues.<br>
connection_type = "Localtunnel"  #@param ["Localtunnel","NGROK","NGROK_APT"]
ngrok_token = "None" #@param ['None'] {allow-input: true}

In [ ]:
#@title 2. Models { display-mode: "form" }

#@markdown ###All model mangement is all done in-app, the "Model Manager" is found on the left hand side once you are in the app. <br />
#@markdown If you are using Temprory Mode, would you like to attach your Google Drive purely to import models from?
GDrive_Import = "No" #@param ["Yes","No"]

#@markdown The path to the root of your Google drive will be added as: /content/drive/MyDrive/ You will use that in the "Model manager" to import models.

In [ ]:
#@title 3. Advanced Options { display-mode: "form" }
#@markdown ##These settings have all been removed. See below for details.

#@markdown ##Version
#@markdown Since Version 5.6.0+ Version control has been non-existant, this app updates on EVERY RUN now.

#@markdown ---

#@markdown #Model training.<br />
#@markdown "hollowstrawberry" has an amazing Google colab LoRA maker, it is 100X Better than I could do! It can be found here:<br />
#@markdown Dataset Maker - https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Dataset_Maker.ipynb <br />
#@markdown LoRA Trainer - https://colab.research.google.com/github/hollowstrawberry/kohya-colab/blob/main/Lora_Trainer.ipynb

In [ ]:
#@title 4. Build and Configure. { display-mode: "form" }
#@markdown Install InvokeAI and configure it as requested, This takes around 2-4 mins.

from IPython.display import clear_output

#Setting File Path
file_path = '/content/invokeai'

#Set up temporary storage if running in "Temporary" mode.
if Type == "Temporary":
    import os
    if not os.path.exists(file_path):
      os.makedirs(file_path)

    #Mount google drive for model imports if requested.
    if GDrive_Import == "Yes":
      import os
      from google.colab import drive
      if not os.path.exists('/content/drive/'):
        drive.mount('/content/drive')

# Mount and set up Google drive if running in "Google_Drive" mode.
if Type == "Google_Drive":
  import os
  from google.colab import drive
  if not os.path.exists(file_path):
    drive.mount('/content/drive')
    if not os.path.exists(file_path):
      os.makedirs(file_path)

#set working DIR
%cd {file_path}

#Clear Output
clear_output()

print("=======================")
print("|Install Dependencies.|");
print("=======================")

#Update pip
!curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!python -m pip install --upgrade pip

#Install UV
!pipx install uv

#Create InvokeAI root
import os
os.environ['INVOKEAI_ROOT'] = file_path
if not os.path.exists(file_path):
  os.makedirs(invokeai_root)

#Confirm that This is working in the correct place.
%cd {file_path}

#Clear Output
clear_output()

print("================================================================================")
print("|This is the first loading process for the App. It takes a few seconds to load.|");
print("================================================================================")
#%cd {file_path}
!uv venv --relocatable --prompt invoke --python 3.11 --python-preference only-managed .venv
!source .venv/bin/activate
print(" ")
print("====================================================================================================================================================================================")
print("|This is the second loading process for the App. It takes on average 2-3 mins to load and has no indication of loading, for some reason if it displays the progress, Colab crashes!|");
print("====================================================================================================================================================================================")
!uv pip install invokeai[xformers] --python 3.11 --python-preference only-managed --force-reinstall &> /dev/null

#This 'Deactivate' is needed incase you run this twice in one session where an update is released between runs, it will error 99% of the time.
if os.path.exists("/content/RunSuccess"):
  print("This Deactivate only is needed on subsiquent runs. Ignore that it has errored. It will most of the time, and is a cleanup.")
  !deactivate
  print("Deactivate command has been executed.")
if not os.path.exists("/content/RunSuccess"):
  !touch "/content/RunSuccess"

!source .venv/bin/activate

if Type == "Google_Drive":
  print(" ")
  print("==================================================================================================")
  print("|This is the third loading process for the App, setting the program up for 'Google Drive' storage|");
  print("==================================================================================================")
  !echo "# Internal metadata - do not edit:" > /content/invokeai/invokeai.yaml
  !echo "schema_version: 4.0.2" >> /content/invokeai/invokeai.yaml
  !echo " " >> /content/invokeai/invokeai.yaml
  !echo "# Put user settings here - see https://invoke-ai.github.io/InvokeAI/configuration/:" >> /content/invokeai/invokeai.yaml
  !echo "models_dir: /content/drive/MyDrive/InvokeAI/Models" >> /content/invokeai/invokeai.yaml
  !echo "outputs_dir: /content/drive/MyDrive/InvokeAI" >> /content/invokeai/invokeai.yaml
  !echo "db_dir: /content/drive/MyDrive/InvokeAI/DB" >> /content/invokeai/invokeai.yaml
  !echo "enable_partial_loading: true" >> /content/invokeai/invokeai.yaml
  print("Locations set for Google Drive!");

if Type == "Temporary":
  print(" ")
  print("===============================================================================================")
  print("|This is the third loading process for the App, setting the program up for 'Temporary' storage|");
  print("===============================================================================================")
  !echo "# Internal metadata - do not edit:" > /content/invokeai/invokeai.yaml
  !echo "schema_version: 4.0.2" >> /content/invokeai/invokeai.yaml
  !echo " " >> /content/invokeai/invokeai.yaml
  !echo "# Put user settings here - see https://invoke-ai.github.io/InvokeAI/configuration/:" >> /content/invokeai/invokeai.yaml
  !echo "enable_partial_loading: true" >> /content/invokeai/invokeai.yaml
  print("Locations set for Temporary Storage!");

print(" ")
print("=============================")
print("|Done! Launch the app below!|")
print("=============================")

In [ ]:
#@title 5. Start InvokeAI. { display-mode: "form" }
#@markdown ## Starting the App
#@markdown This step takes about 15 seconds to generate your URL but 30 seconds after it is launched before it will work fully! <br>

#@markdown ## Notes about connection types.
#@markdown **NGROK** (Recomneded) = Very stable but requires a token, see the "configuration" step for more details.<br>
#@markdown **NGROK_APT** = An alternate version of NGROK that runs using the system packages rather than Python. <br>
#@markdown **Localtunnel** = Once it gets going it is quite stable but often has "502" Errors, You must wait for THEM to fix it, please try another connection type.<br>
#@markdown **Serveo** = Removed, No longer Exists.

!source .venv/bin/activate

%cd {file_path}
import os

#Clear Output
clear_output()

#Serveo is broken Keeping this code incase it comes back.
#if connection_type == "Serveo":
  #!ssh -o StrictHostKeyChecking=no -o ServerAliveInterval=60 -R 80:localhost:9090 serveo.net & ! . {file_path}/.venv/bin/activate; invokeai-web

if connection_type == "Localtunnel":
  print("How to connect to localtunnel:");
  print("A localtunnel Interface connection is generated here, To use this, please do the following ")
  print("1. Copy this IP address")
  !curl ipv4.icanhazip.com
  print("2. Click the random 'https://XXX-YYY-ZZZ.loca.lt' link that is generated below.")
  print("3. Paste the IP into the provided box and submit. ")
  print(" ")
  print("Note: An error of '502 Bad Gateway' typically is an error at Localtunnels end. A '504 Gateway Time-out' Error means invokeai has not started yet.")
  print(" ")
  !npm install -g localtunnel
  !lt --port 9090 & ! . {file_path}/.venv/bin/activate; invokeai-web


#NGROK_Python
if connection_type == "NGROK":
  if ngrok_token == "None":
    print("You have Selected NGROK but did not supply an NGROK token.")
    print("Falling back to a 'Localtunnel' connection type.")
    print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Servio'.")
    connection_type = "Localtunnel"
  if ngrok_token != "None":
    !pip install pyngrok --quiet
    from pyngrok import ngrok
    ngrok.kill()
    ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(9090).public_url
    print(f'InvokeAI Public URL: {public_url}')
    ! . {file_path}/.venv/bin/activate; invokeai-web

#NGROK_APT
if connection_type == "NGROK_APT":
  if ngrok_token == "None":
    print("You have Selected NGROK but did not supply an NGROK token.")
    print("Falling back to a 'Localtunnel' connection type.")
    print("Please either add an NGROK token to step 1, re-run step 1, then re-run this step, or just re-run this step to use 'Servio'.")
    connection_type = "Localtunnel"
  if ngrok_token != "None":
    !curl -sSL https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null
    !echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list
    !sudo apt update
    !sudo apt install ngrok
    !ngrok config add-authtoken {ngrok_token}
    clear_output()
    !echo "You can find the connection URL here in the NGROK portal:"
    !echo "https://dashboard.ngrok.com/endpoints"
    !nohup ngrok http http://localhost:9090 &
    ! . {file_path}/.venv/bin/activate; invokeai-web

# FAQ:

## Images are taking a LONG time to generate. (~30 mins for a 512x512 image.)
You may have used up your computer allowance and are running in CPU mode. While this is running click the "RAM / Disk" button in the top right to see if it lists "GPU RAM" If not, you are not connected to a GPU instance.
or maybe, you have the GPU turned off, check "Edit > Notebook settings" to make sure a GPU is selected. ("T4 GPU" is the current default)

## It is taking a while to even start loading.
If you have started a new instance / are changing a model. the model needs to be loaded into Invoke. If you are using "Google Drive" It has to download the model to the runtime, this can take a while especially for XL (5-10 mins) or FLUX models (20+ mins)

## I get an error "You have recently exceeded an allowance, most recently at (Time)"
This is typically when you have exceeded some Google restriction(s). This is most common when you stop / start instances frequently and often does not affect anything. <br>
If it is blocking things, you typically have to wait 1 hour for soft allowances to reset.  

##How much longer can my instance run for today?
If you click the RAM / DISK" button in the top right it will tell you something like "At your current usage level, this runtime may last up to 1 hour 20 minutes"

## My instance disconnects while I'm using it.
Technically using Web interface to run programs in a way like this project does, is against Google Collab's Terms of Service. If you don't look at this tab for 15+ mins it may hibernate. Simply flick back to this tab on a semi-regular basis to keep it active!

## Are there any known issues?
If there are, I will typically update the header at the top of this page above the "Introduction".

## I want to make my own changes to this file, and save them.
Feel free to! "File > Save a copy in drive" is the safest way to do this.

## Is this maintained frequently?
I am a solo dev that maintains it the best that I can, I do use this project frequently so hopefully I will notice any major issues. <br>
If you have any issues you can raise them on Github I will deal with them when I can! <br>
https://github.com/MikeEmmett/InvokeAI/issues

## I am using Google Drive and I want to reset everything.
If you log onto https://drive.google.com/drive/my-drive with your Google account you will find a folder called "InvokeAI" <br>
If you just want to reset it without keeping anything, simply delete the whole folder. <br>
If you want to reset it but keep the models / Images in an archive, rename the folder.

## I don't understand the options above...
Experiment! This will work fully if you just do "Runtime > Run all" It will connect to your Google Drive and spin up a "Localtunnel" connection and just simply copy the IP address / paste into the link as instructed in the last step! Also feel free to play around, you can't break things permanently!